# Data analyse CLL

#### Definitions
CTL = reference simulation without turbines, the control run\
WFP = wind farm parametrization

#### Packages

In [ ]:
#Import packages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import tarfile
import io
import csv
import pandas as pd
# import netCDF4
# import h5netcdf
# import glob

#### Functions

In [ ]:
#Function to read (csv) data file
def read_data(filename):
    headers = ['lon','lat','type','r','z']
    df = pd.read_csv(filename,skiprows=1,names= headers,delim_whitespace=True)
    return df

In [ ]:
#Function to read (nc) data file
def nc_read(directory):
    data = xr.open_mfdataset(directory)
    return data

In [ ]:
#Read data from the coordinates of the wind turbines
turbines = read_data('E:\data\wind_turbine_coordinates_2020_2050.tab')

In [ ]:
#Opening the mask, since the mask is saved in a netCDF file
directory_mask = "C:/Users/esmer/OneDrive/TN jaar 3/BEP/mask.nc"
mask = nc_read(directory_mask)

In [ ]:
#Opening the mask for wind turbines of type 900 and 901
directory_mask = "C:/Users/esmer/OneDrive/TN jaar 3/BEP/mask_type900901.nc"
mask_type900901 = nc_read(directory_mask)

In [ ]:
#Opening the mask for all wind turbines on sea
directory_mask = "C:/Users/esmer/OneDrive/TN jaar 3/BEP/masksea.nc"
mask_sea = nc_read(directory_mask)

#### Determine the different seasons of the CLL: spring, summer, autumn, winter

In [ ]:
#The directories for the different seasons
#Spring, summer, autumn, winter
directory1 = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc"]

directory2 = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc"]

# Qualitative approach

## Difference in CLL

#### Determine the minimum and maximum of the difference to find the boundaries of the colorbar

In [ ]:
min = []
max = []
for i in range(4):    
    CLLwfp = nc_read(directory1[i])
    CLLctl = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLctl = CLLctl.mean(dim='time')
    diffCLL = avgCLLwfp - avgCLLctl
    minimum = np.min(diffCLL.cll.values)
    maximum = np.max(diffCLL.cll.values)
    min = np.append(min,minimum)
    max = np.append(max,maximum)
    
print(min)
print(max)

In [ ]:
min = []
max = []
for i in range(4):    
    CLLwfp = nc_read(directory1[i])
    CLLctl = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLctl = CLLctl.mean(dim='time')
    diffCLL = avgCLLwfp - avgCLLctl
    minimum = np.min(avgCLLwfp.cll.values)
    maximum = np.max(avgCLLctl.cll.values)
    min = np.append(min,minimum)
    max = np.append(max,maximum)
    
print(min)
print(max)

### For one year (2020)

In [ ]:
#Determining the values of CLL for one year instead of seasons
CTL=[]
WFP = []
for i in range(4):
    CTL.append(directory2[i])
    WFP.append(directory1[i])
    
#Extracting the data from CTL into an array cll
ctl = []
wfp = []
_ctl = []
_wfp = []
for i in range(4):
    _ctl.append(nc_read(CTL[i]))
    combined_ctl = xr.concat(_ctl,dim='time')
    _wfp.append(nc_read(WFP[i]))
    combined_wfp = xr.concat(_wfp,dim='time')


# combined_ctl = xr.concat(_ctl,dim='time')
# combined_wfp = xr.concat(_wfp,dim='time')
# #Now WFP gives the data in an xarray for one year
#Now CTL gives the data in an xarray for one year

#### Low cloud cover for year 2020 in CTL

In [ ]:
#For loop to plot the difference in CLL with and without wind farms
#plt.figure(dpi=1200)
avgCLLwfp = combined_wfp.mean(dim='time')
avgCLLctl = combined_ctl.mean(dim='time')
diffCLL = avgCLLwfp - avgCLLctl
plt.pcolormesh(avgCLLctl.lon,avgCLLctl.lat,avgCLLctl.cll,cmap='Spectral_r',vmin=0,vmax=1)
plt.colorbar()
plt.scatter(turbines.lon,turbines.lat,s=0.5,c='k',alpha=0.1)
plt.xlabel('Longitude(°)')
plt.ylabel('Latitude(°)')
# plt.title('The difference of low cloud area fraction')
plt.xlim(3.3,10.3)
plt.ylim(53,56.6)
plt.savefig('clldiff2020.jpg',dpi=300) #save the figure
plt.show()

In [ ]:
#The average CLL above wind turbines of type 900 and 901.
mask = avgCLLctl.cll.values*mask_type900901.__xarray_dataarray_variable__.values
sumCLL = np.sum(mask)
sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
avgCLLdiff = sumCLL/sumturbines
avgCLLdiff

In [ ]:
#The average CLL in 2020
np.mean(avgCLLctl.cll.values)

#### Difference in cloud cover between WFP and CTL for 2020

In [ ]:
#The difference in CLL with and without wind farms
#plt.figure(dpi=1200)
avgCLLwfp = combined_wfp.mean(dim='time')
avgCLLctl = combined_ctl.mean(dim='time')
diffCLL = avgCLLwfp - avgCLLctl
plt.pcolormesh(diffCLL.lon,diffCLL.lat,diffCLL.cll,cmap='Spectral_r',vmin=-0.07,vmax=0.07)
plt.colorbar()
plt.scatter(turbines.lon,turbines.lat,s=0.5,c='k',alpha=0.1)
plt.xlabel('Longitude(°)')
plt.ylabel('Latitude(°)')
# plt.title('The difference of low cloud area fraction')
plt.xlim(3.3,10.3)
plt.ylim(53,56.6)
plt.savefig('clldiff2020.jpg',dpi=300) #save the figure
plt.show()

In [ ]:
#The average difference in CLL above wind turbines of type 900 and 901.
mask = diffCLL.cll.values*mask_type900901.__xarray_dataarray_variable__.values
sumCLL = np.sum(mask)
sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
avgCLLdiff = sumCLL/sumturbines
avgCLLdiff

In [ ]:
np.max(diffCLL.cll.values)

### Per season

#### Difference in cloud cover between WFP and CTL per season

In [ ]:
#For loop to plot the difference in CLL with and without wind farms
#plt.figure(dpi=1200)
for i in range(4):
    CLLwfp = nc_read(directory1[i])
    CLLctl = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLctl = CLLctl.mean(dim='time')
    diffCLL = avgCLLwfp - avgCLLctl
    plt.pcolormesh(diffCLL.lon,diffCLL.lat,diffCLL.cll,cmap='Spectral_r',vmin=-0.07,vmax=0.07)
    plt.colorbar()
    plt.scatter(turbines.lon,turbines.lat,s=0.5,c='k',alpha=0.1)
    plt.xlabel('Longitude(°)')
    plt.ylabel('Latitude(°)')
#     plt.title('The difference of low cloud area fraction')
    plt.xlim(3.3,10.3)
    plt.ylim(53,56.6)
    #plt.savefig('cllSpringdiff.jpg',dpi=300) #save the figure
    plt.show()

## CLL with WFP

In [ ]:
#For loop to plot a figure with WFP
#plt.figure(dpi=1200)
for i in range(4):
    CLLwfp = nc_read(directory1[i])
    CLLref = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLref = CLLref.mean(dim='time')
    #plt.subplot(2,2,1+i)
    plt.pcolormesh(avgCLLwfp.lon,avgCLLwfp.lat,avgCLLwfp.cll,cmap='Spectral_r',vmin=0,vmax=1)
    plt.colorbar()
    plt.scatter(turbines.lon,turbines.lat,s=0.5,c='k',alpha=0.1)
    plt.xlabel('Longitude(°)')
    plt.ylabel('Latitude(°)')
    plt.title('Low cloud area fraction with WFP')
    plt.xlim(3.3,10.3)
    plt.ylim(53,56.6)
    #plt.savefig('cllSpringdiff.jpg',dpi=300) #save the figure
    plt.show()

## CLL without WFP

#### Over the seasons

In [ ]:
#For loop to plot CLL without WFP
#plt.figure(dpi=1200)
for i in range(4):
    CLLwfp = nc_read(directory1[i])
    CLLref = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLref = CLLref.mean(dim='time')
    #plt.subplot(2,2,1+i)
    plt.pcolormesh(avgCLLref.lon,avgCLLref.lat,avgCLLref.cll,cmap='Spectral_r',vmin=0,vmax=1)
    plt.colorbar()
    plt.scatter(turbines.lon,turbines.lat,s=0.5,c='k',alpha=0.1)
    plt.xlabel('Longitude(°)')
    plt.ylabel('Latitude(°)')
#     plt.title('Low cloud area fraction without WFP')
    plt.xlim(3.3,10.3)
    plt.ylim(53,56.6)
    #plt.savefig('cllSpringdiff.jpg',dpi=300) #save the figure
    plt.show()
    print(np.mean(avgCLLref.cll.values))

# Quantitative approach

## Using the mask for calculating CLL for wind turbines type 900 and 901

Hierboven hebben we gekeken naar het verschil in low cloud coverage met en zonder windmolens. Dit hebben we kwalitatief gedaan door te plotten hoe de CLL verandert als er wel of geen windmolens staan bij de noordzee. Maar nu willen we er kwantitatief naar kijken. Ik heb een mask met nullen en enen gemaakt en die mask is een 1 als er een windmolen van type 900 en 901 aanwezig is, wat de grootste windmolen is en daarmee ook de grootste impact op het klimaat en specifiek wolkenvorming zal hebben, en een 0 als er geen windmolen van type 901 aanwezig is. 

We gaan nu CLL vermenigvuldigen met de mask. Dat gaan we doen met een zonder windmolens, dus de CTL en de WFP variabele. Hier willen we vervolgens het verschil tussen weten en met dat verschil willen we dat normaliseren door te delen over de som van de windturbines van type 901.

#### Directories CLL for seasons

In [ ]:
#The directories for the different seasons
#Spring, summer, autumn, winter
directory1 = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.2020*.1hr.nc"]

directory2 = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.2020*.1hr.nc"]

#### Kwantitatief verschil in CLL bepalen voor 1 windmolen van type 900 en 901

The difference in CLL above wind turbines of type 900 and 901 between CTL and WFP per season

In [ ]:
#Opening the mask, since the mask is saved in a netCDF file
#Importing the mask of wind turbines 900 and 901
directory_mask = "C:/Users/esmer/OneDrive/TN jaar 3/BEP/mask_type900901.nc"
mask_type900901 = nc_read(directory_mask)

#We gaan voor 4 verschillende seizoenen bepalen wat het gemiddelde verschil in CLL is tussen CTL en WFP
CLL = []
for i in range(4): 
    CLLwfp = nc_read(directory1[i])
    CLLctl = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim="time")
    avgCLLctl = CLLctl.mean(dim="time")
    diffCLL = avgCLLwfp - avgCLLctl
    diff = diffCLL.cll.values*mask_type900901.__xarray_dataarray_variable__.values
    sumCLL = np.sum(diff)
    sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
    avgdiffCLL = sumCLL/sumturbines
    CLL = np.append(CLL,avgdiffCLL)    
print(CLL)

Average values of CLL above wind turbines per season

In [ ]:
CLLvalues=[]
for i in range(4):
    CLLctl = nc_read(directory2[i])
    CLLctlmean = CLLctl.mean(dim='time')
    CLLctlmean.cll.load()
    
    maskcllctl = CLLctlmean.cll.values*mask_type900901.__xarray_dataarray_variable__.values
    sumcllmask = np.sum(maskcllctl)
    sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
    valuescll = sumcllmask/sumturbines
    CLLvalues.append(valuescll)

In [ ]:
CLLvalues

In [ ]:
CTL=[]
WFP = []
for i in range(4):
    CTL.append(directory2[i])
    WFP.append(directory1[i])
    
#Extracting the data from CTL into an array cll
ctl = []
wfp = []
for i in range(4):
    _ctl = []
    _ctl.append(nc_read(CTL[i]))

    
    _wfp = []
    _wfp.append(nc_read(WFP[i]))


combined_ctl = xr.concat(_ctl,dim='time')
combined_wfp = xr.concat(_wfp,dim='time')
# #Now WFP gives the data in an xarray for one year
#Now CTL gives the data in an xarray for one year

In [ ]:
a = combined_wfp.mean(dim = "time") - combined_ctl.mean(dim="time")
mask = a.cll.values*mask_type900901.__xarray_dataarray_variable__.values
sumCLL = np.sum(mask)
sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
avgCLLdiff = sumCLL/sumturbines

avgCLLdiff
np.max(a.cll.values)

In [ ]:
mask = avgCLLctl.cll.values*mask_type900901.__xarray_dataarray_variable__.values
sumCLL = np.sum(mask)
sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
avgCLLdiff = sumCLL/sumturbines

In [ ]:
# avgCLLwfp = combined_wfp.mean(dim='time')
avgCLLctl = _ctl[0].mean(dim="time")
# avgCLLctl = combined_ctl.mean(dim='time')
avgCLLwfp = _wfp[0].mean(dim='time')
diffCLL = avgCLLwfp - avgCLLctl 
diff = diffCLL.cll.values*mask_type900901.__xarray_dataarray_variable__.values
sumCLL = np.sum(diff)
sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
avgdiffCLL = sumCLL/sumturbines
CLL = avgdiffCLL  

CLL

## Calculating total difference in CLL

Now we want to know whether there is a total increase in low cloud coverage or that the distribution of the low clouds changed rather than more CLL.

In [ ]:
CLLavg=[]
for i in range(4):
    CLLwfp = nc_read(directory1[i])
    CLLctl = nc_read(directory2[i])
    avgCLLwfp = CLLwfp.mean(dim='time')
    avgCLLctl = CLLctl.mean(dim='time')
    diffCLL = avgCLLwfp - avgCLLctl
    sumcll = np.sum(diffCLL.cll.values) #Sum of the low cloud area fraction between wfp and ctl
    gridpoints = np.ones([146,166]) #The amount of grid points
    sumgridpoints = np.sum(gridpoints)
    avgcll = sumcll/sumgridpoints
    CLLavg = np.append(CLLavg,avgcll)
print(CLLavg)

In [ ]:
np.mean(CLLavg)

In [ ]:
CLL = []
for i in range(366):
    diffcll = dayswfp[i] - days[i]
    diff = diffcll.cll.values*mask_type900901.__xarray_dataarray_variable__.values
    sumCLL = np.sum(diff)
    sumturbines = np.sum(mask_type900901.__xarray_dataarray_variable__.values)
    avgdiffCLL = sumCLL/sumturbines
    CLL = np.append(CLL,avgdiffCLL) 

np.mean(CLL)

# Analysis of bins of CLL

#### CTL

In [ ]:
directory = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202001*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202002*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202003*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202004*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202005*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202006*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202007*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202008*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202009*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202010*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202011*.1hr.nc",
             "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterCTL/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_CTL_ptA.202012*.1hr.nc"
            ]

#### WFP

In [ ]:
directorywfp = ["E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202001*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202002*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202003*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202004*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSpringWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202005*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202006*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202007*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLSummerWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202008*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202009*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202010*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLFallWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202011*.1hr.nc",
                "E:/data/transfer_1722109_files_fe942ebb/CLL/CLLWinterWFP/cll.his.GERMAN_BIGHT.WINS50_43h21_fERA5_WFP_2050_ptA.202012*.1hr.nc",
               ]

### CTL: calculation CLL above all wind turbines (900 and 901) and all days in 2020

In [ ]:
#Array for different months
#Without WFP
CTL = []
for i in range(12):
    CTL.append(directory[i])

In [ ]:
#Extracting the data from CTL into an array cll
cll = []
for i in range(12):
    _ctl = []
    _ctl.append(nc_read(CTL[i]))
    combined_ctl = xr.concat(_ctl,dim='time')
    cll.append(combined_ctl)

In [ ]:
#Average over all hours in one day
days = []
dayssel=[]
months = [32,30,32,31,32,31,32,32,31,32,31,32]
j=0
for i in range(12): 
    for j in range(1,months[i]): #All days in year 2020
        dayssel = cll[i].isel(time=(cll[i].time.dt.day == j)) #Selecting the days
        daycll = dayssel.mean(dim='time') #Calculating the mean CLL over that day
        days.append(daycll)
    j+=1

In [ ]:
#Loading CLL data for all days
for i in range(len(days)):
    days[i].cll.load()

In [ ]:
indices = np.where(mask_type900901.__xarray_dataarray_variable__.values == 1) 

In [ ]:
cllvaluesctl=[]
for i in range(366):
    clldayctl=[]
    for j in range(1570):
        clldayctl.append(days[i].cll.values[indices[0][j],indices[1][j]])
    cllvaluesctl.append(clldayctl)

In [ ]:
pd.DataFrame(cllvaluesctl)

In [ ]:
#Array to determine the value of CLL for a specific day above the wind turbines
avgcllday = []
for i in range(366):
    cllsum = np.sum(cllvaluesctl[i])
    turbines = 1570
    avgcllday.append(cllsum/turbines)

In [ ]:
pd.DataFrame(avgcllday)

### WFP: calculation CLL above all wind turbines (900 and 901) and all days in 2020

In [ ]:
#Array for different months
WFP = []
for i in range(12):
    WFP.append(directorywfp[i])

In [ ]:
wfp = []
for i in range(12):
    _wfp = []
    _wfp.append(nc_read(WFP[i]))
    combined_wfp = xr.concat(_wfp,dim='time')
    wfp.append(combined_wfp)

In [ ]:
#Average over all hours in one day
dayswfp = []
dayssel=[]
months = [32,30,32,31,32,31,32,32,31,32,31,32]
j=0
for i in range(12): 
    for j in range(1,months[i]): #All days in year 2020
        dayssel = wfp[i].isel(time=(wfp[i].time.dt.day == j)) #Selecting the days
        daywfp = dayssel.mean(dim='time') #Calculating the mean CLL over that day
        dayswfp.append(daywfp)
    j+=1

In [ ]:
for i in range(len(dayswfp)):
    dayswfp[i].cll.load()

In [ ]:
cllvalueswfp=[]
for i in range(366):
    clldaywfp=[]
    for j in range(1570):
        clldaywfp.append(dayswfp[i].cll.values[indices[0][j],indices[1][j]])
    cllvalueswfp.append(clldaywfp)

In [ ]:
#Array to determine the average value of CLL above all wind turbines
avgclldaywfp = []
for i in range(366):
    cllsum = np.sum(cllvalueswfp[i])
    turbines = 1570
    avgclldaywfp.append(cllsum/turbines)

In [ ]:
pd.DataFrame(avgclldaywfp)

### Dividing CLL above wind turbines in bins

In [ ]:
#Calculation in which bins the values of CLL for ctl belong and the same values for WFP for that case
cllctl = []
cllwfp = []
bound = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
for j in range(366): #All days
    matrixctl = []
    matrixwfp = []
    for k in range(len(bound)-1):
        if bound[k] <= avgcllday[j] < bound[k+1]: #Does cll of ctl of a certain day belong to bound [k,k+1]
            matrixctl.append(avgcllday[j]) #If so, append the value of cll to matrixctl
            matrixwfp.append(avgclldaywfp[j]) #Append the value of the same day in the same bin of wfp
        else:
            matrixctl.append(np.nan)
            matrixwfp.append(np.nan)
    cllctl.append(matrixctl)
    cllwfp.append(matrixwfp)
    
#cllctl is an array containing the values of cll of ctl in the bins
#cllwfp is an array containing the values of cll of wfp in the bins of ctl
#This means you get, for every day, the average value of CLL in a bin. 10 bins and for every day one value of CLL in
# one bin.

In [ ]:
#The difference in CLL between wfp and ctl
diffcll = np.subtract(cllwfp,cllctl)

In [ ]:
pd.DataFrame(diffcll)

Now we know at a certain day, what bin the value of CLL belongs to and what the difference of that day and that bin is between WFP and CTL. Next, we want to know what the average CLL is over a bin. For this, we need to know the sum of the values of CLL in a bin and the sum of the turbines inside that bin.

In [ ]:
pd.DataFrame(diffcll)

In [ ]:
#Function to calculate the sum of the turbines
def numberOfNonNans(data):
    count = 0
    for i in data:
        if not np.isnan(i):
            count += 1
    return count 

In [ ]:
#Make an array of the sum of the turbines for the different bins
sumturbines=[]
for i in range(10):
    sumturbines.append(numberOfNonNans(diffcll[:,i]))

np.sum(sumturbines)
sumturbines

Now we want to take the average over the bins.

In [ ]:
cllbin=[]
for j in range(10):
    sumdiffcll = np.nansum(diffcll[:,j])
    cllbin.append(sumdiffcll/sumturbines[j])
    
cllbin       

### Seasons

In [ ]:
#The difference in CLL between wfp and ctl
diffcll = np.subtract(cllwfp,cllctl)

In [ ]:
pd.DataFrame(diffcll)

#### Calculation of the different bins and values of CLL

In [ ]:
# #Spring
diffspring = diffcll[60:153]

#Make an array of the sum of the turbines for the different bins
sumturbinesspring=[]
for i in range(10):
    sumturbinesspring.append(numberOfNonNans(diffspring[:,i]))

cllbinspring=[]
for j in range(10):
    sumdiffcll = np.nansum(diffspring[:,j])
    if sumdiffcll == 0:
        cllbinspring.append(0)
    else:
        cllbinspring.append(sumdiffcll/sumturbinesspring[j])

# #Summer
diffsummer = diffcll[153:245]

#Make an array of the sum of the turbines for the different bins
sumturbinessummer=[]
for i in range(10):
    sumturbinessummer.append(numberOfNonNans(diffsummer[:,i]))

cllbinsummer=[]
for j in range(10):
    sumdiffcll = np.nansum(diffsummer[:,j])
    if sumdiffcll == 0:
        cllbinsummer.append(0)
    else:
        cllbinsummer.append(sumdiffcll/sumturbinessummer[j])
    
# #autumn
diffautumn = diffcll[245:336]

#Make an array of the sum of the turbines for the different bins
sumturbinesautumn=[]
for i in range(10):
    sumturbinesautumn.append(numberOfNonNans(diffautumn[:,i]))

cllbinautumn=[]
for j in range(10):
    sumdiffcll = np.nansum(diffautumn[:,j])
    if sumdiffcll == 0:
        cllbinautumn.append(0)
    else:
        cllbinautumn.append(sumdiffcll/sumturbinesautumn[j])

#Winter
diffwinter1 = diffcll[:60]
diffwinter2 = diffcll[336:366]
diffwinter1 = pd.DataFrame(diffwinter1)
diffwinter2 = pd.DataFrame(diffwinter2)
diffwinter = diffwinter1.append(diffwinter2)
diffwinter = np.array(diffwinter)

sumturbineswinter=[]
for i in range(10):
    sumturbineswinter.append(numberOfNonNans(diffwinter[:,i]))

cllbinwinter=[]
for j in range(10):
    sumdiffcll = np.nansum(diffwinter[:,j])
    if sumdiffcll == 0:
        cllbinwinter.append(0)
    else:
        cllbinwinter.append(sumdiffcll/sumturbinesautumn[j])
# diffwinter

#### Calculation of frequency and probability

In [ ]:
#Function to calculate the sum of the turbines
def numberOfNonNans(data):
    count = 0
    for i in data:
        if not np.isnan(i):
            count += 1
    return count 

In [ ]:
#Make an array of the sum of the turbines for the different bins
sumturbines=[]
for i in range(10):
    sumturbines.append(numberOfNonNans(diffcll[:,i]))

np.sum(sumturbines)
sumturbines

In [ ]:
cllbin=[]
for j in range(10):
    sumdiffcll = np.nansum(diffcll[:,j])
    cllbin.append(sumdiffcll/sumturbines[j])
    
cllbin   

In [ ]:
probspring = np.divide(sumturbinesspring,np.sum(sumturbinesspring))
probsummer = np.divide(sumturbinessummer,np.sum(sumturbinessummer))
probautumn = np.divide(sumturbinesautumn,np.sum(sumturbinesautumn))
probwinter = np.divide(sumturbineswinter,np.sum(sumturbineswinter))

probspring

In [ ]:
a=[]
a.append(np.sum(probspring*cllbinspring))
a.append(np.sum(probsummer*cllbinsummer))
a.append(np.sum(probautumn*cllbinautumn))
a.append(np.sum(probwinter*cllbinwinter))

a

In [ ]:
np.max(cllbin)

In [ ]:
bins = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
plt.plot(bins,cllbin,label="All days",marker='o',color='orangered')
# plt.plot(bins,cllbinspring,label="spring",marker='o',color='darkorange')
# plt.plot(bins,cllbinsummer,label="summer",marker='o',color='gold')
# plt.plot(bins,cllbinautumn,label="autumn",marker='o',color='limegreen')
# plt.plot(bins,cllbinwinter,label="winter",marker='o',color='lightblue')
# plt.xticks(bins,values)
values = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
# plt.grid(axis='x',linewidth=0.5)
grid_points = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
plt.xticks(grid_points)
plt.grid(axis='x')
# plt.yticks([0.0])
plt.grid(axis='y')
# plt.legend()
# plt.title("Difference in cloud cover")
plt.xlabel("Low cloud cover CTL")
plt.ylabel("Difference in low cloud cover")
plt.savefig('differencebinsCLL.jpg',dpi=300) #save the figure
plt.show()

In [ ]:
bins = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
# plt.plot(bins,cllbin,label="All days",marker='o',color='orangered')
plt.plot(bins,cllbinspring,label="spring",marker='o',color='darkorange')
plt.plot(bins,cllbinsummer,label="summer",marker='o',color='gold')
plt.plot(bins,cllbinautumn,label="autumn",marker='o',color='limegreen')
plt.plot(bins,cllbinwinter,label="winter",marker='o',color='lightblue')
# plt.xticks(bins,values)
values = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
# plt.grid(axis='x',linewidth=0.5)
grid_points = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
plt.xticks(grid_points)
plt.grid(axis='x')
# plt.yticks([0.0])
plt.grid(axis='y')
plt.legend()
# plt.title("Difference in cloud cover")
plt.xlabel("Fraction of low cloud cover CTL")
plt.ylabel("Difference in fraction of low cloud cover")
plt.savefig('differencebinsCLL.jpg',dpi=300) #save the figure
plt.show()

### Calculating the frequency and probability of a bin

#### Frequency

In [ ]:
frequency = sumturbines

In [ ]:
bins = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
plt.plot(bins,frequency,label="All days",color='orangered',drawstyle='steps-mid')
grid_points = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
plt.xticks(grid_points)
plt.grid(axis='x')
# plt.yticks([0.0])
plt.grid(axis='y')
plt.xlabel("Bins")
plt.ylabel("Frequency")
# plt.title("The amount of days with average CLL inside bin")
plt.legend()
plt.savefig('HistogramfrequencyCLL.jpg',dpi=300) #save the figure
plt.show()
#The amount of days that the average CLL over all wind turbines of 900 and 901 is inside a bin

#### Probability

In [ ]:
prob = np.divide(frequency,366)

In [ ]:
bins = [0.05,0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95]
plt.plot(bins,prob,label="All days",color='orangered',drawstyle='steps-mid')
# plt.plot(bins,probspring,label="Spring",drawstyle='steps-mid')
# plt.plot(bins,probsummer,label="Summer",drawstyle='steps-mid')
# plt.plot(bins,probautumn,label="Autumn",drawstyle='steps-mid')
# plt.plot(bins,probwinter,label="Winter",drawstyle='steps-mid')
grid_points = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
plt.xticks(grid_points)
plt.grid(axis='x')
# plt.yticks([0.0])
plt.grid(axis='y')
plt.xlabel("Bins")
plt.ylabel("Probability")
plt.legend()
plt.title("The probability that CLL is inside a bin")
plt.savefig('probabilitycllbin.jpg',dpi=300) #save the figure

In [ ]:
values = prob * cllbin
values
np.sum(values[1:])